In [0]:
#Installing relevant packages
!pip install --upgrade pip
!pip install yfinance==0.1.67
!mamba install bs4==4.10.0 -y
!pip install beautifulSoup4
!pip install html5lib==1.1 -y

/bin/bash: mamba: command not found

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


In [0]:
!pip install html5lib

In [0]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import html5lib

In [0]:
#The make_graph function takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2022-08-24']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2022-06-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [0]:
#Using the Ticker function Apple stock data is extracted to create a ticker object, its ticker symbol is AAPL. The history function is used to extract stock information from the ticker object, period is set to 10years.
Apple = yf.Ticker('AAPL')
apple_data = Apple.history(period = '10y')
apple_data.reset_index(inplace=True)
apple_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2012-09-21,21.511254,21.592716,21.417848,21.440205,571589200,0.0,0.0
1,2012-09-24,21.035027,21.287989,20.916815,21.155382,639766400,0.0,0.0
2,2012-09-25,21.077907,21.216333,20.610571,20.627110,518789600,0.0,0.0
3,2012-09-26,20.480112,20.601080,20.249200,20.371088,576503200,0.0,0.0
4,2012-09-27,20.343828,20.891401,20.223166,20.865370,594090000,0.0,0.0
...,...,...,...,...,...,...,...,...
2510,2022-09-14,154.789993,157.100006,153.610001,155.309998,87965400,0.0,0.0
2511,2022-09-15,154.649994,155.240005,151.380005,152.369995,90481100,0.0,0.0
2512,2022-09-16,151.210007,151.350006,148.369995,150.699997,162157000,0.0,0.0
2513,2022-09-19,149.309998,154.559998,149.100006,154.479996,81474200,0.0,0.0


In [0]:
#Micrsoft stock information is extracted with same approach
Microsoft = yf.Ticker('MSFT')
Msoft_data = Microsoft.history(period = '10y')
Msoft_data.reset_index(inplace=True)
Msoft_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2012-09-21,25.777526,25.925154,25.498672,25.580688,102348900,0.0,0
1,2012-09-24,25.424853,25.482264,25.129596,25.244419,46825900,0.0,0
2,2012-09-25,25.383842,25.424849,24.899949,24.924553,54266400,0.0,0
3,2012-09-26,24.834345,25.096795,24.637508,24.744127,54672000,0.0,0
4,2012-09-27,24.744127,24.932763,24.514483,24.735926,47129900,0.0,0
...,...,...,...,...,...,...,...,...
2510,2022-09-14,253.529999,254.229996,249.860001,252.220001,23913000,0.0,0
2511,2022-09-15,249.770004,251.399994,244.020004,245.380005,31530900,0.0,0
2512,2022-09-16,244.259995,245.300003,242.059998,244.740005,39775100,0.0,0
2513,2022-09-19,242.470001,245.139999,240.850006,244.520004,26826900,0.0,0


In [0]:
#Requests Library is used to download webpage with Apple revenues
url = "https://www.macrotrends.net/stocks/charts/AAPL/apple/revenue"
apple_html = requests.get(url).text
soup = BeautifulSoup(apple_html, 'html.parser')

In [0]:
#read_html function is used to extract the table with Apple Quarterly Revenue and store it into a dataframe
apple_revenue = pd.read_html(url, match = 'Apple Quarterly Revenue', flavor ='bs4')[0]
apple_revenue.columns=['Date','Revenue']
apple_revenue = apple_revenue[apple_revenue.Date>='2012-08-30']

In [0]:
apple_revenue["Revenue"] = apple_revenue['Revenue'].str.replace(',|\$',"")

<command-2354031396162333>:1: FutureWarning:

The default value of regex will change from True to False in a future version.



In [0]:
#Plotting the Apple Stock Data with make_graph
make_graph(apple_data, apple_revenue, 'Apple')

In [0]:
#Requests Library is used to download webpage with Microsoft Revenues
url = "https://www.macrotrends.net/stocks/charts/MSFT/microsoft/revenue"
Msoft_html = requests.get(url).text
soup = BeautifulSoup(Msoft_html, 'html.parser')

In [0]:
#read_html function is used to extract the table with Microsoft Quarterly Revenue and store it into a dataframe
Msoft_revenue = pd.read_html(url, match = 'Microsoft Quarterly Revenue', flavor ='bs4')[0]
Msoft_revenue.columns=['Date','Revenue']
Msoft_revenue = Msoft_revenue[Msoft_revenue.Date>='2012-08-30']

In [0]:
Msoft_revenue["Revenue"] = Msoft_revenue['Revenue'].str.replace(',|\$',"")

<command-2736002729448596>:1: FutureWarning:

The default value of regex will change from True to False in a future version.



In [0]:
#Plotting the Apple Stock Data with make_graph
make_graph(Msoft_data, Msoft_revenue, 'Microsoft')